In [5]:
import random
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim

from pinn import PINN, Trainer, gradient

In [6]:
def solution_x(t):
    return np.exp(-t)/2 + np.exp(-3*t)/2

def solution_y(t):
    return -np.exp(-t)/2 + np.exp(-3*t)/2

def residual_loss(model, t):
    t.requires_grad = True
    output = model(t)
    x, y = output[:, 0:1], output[:, 1:2]
    x_t = gradient(x, t)        # dx/dt
    y_t = gradient(y, t)        # dy/dt
    residual_x = x_t + 2*x + y
    residual_y = y_t + x + 2*y
    return torch.mean(torch.square(residual_x) + torch.square(residual_y))

In [12]:
## Hyperparameters
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

n_epochs = 10000
learning_rate = 1e-3
layers = [1, 64, 64, 2]

In [ ]:
def to_tensor(x):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if x.ndim == 1:
        return torch.from_numpy(x).float().view(-1, 1).to(device)
    else:
        return torch.from_numpy(x).float().to(device)

def tensor_like(x, value):
    torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return torch.full_like(x, value).to(device)

def to_numpy(x):
    if x.ndim == 2 and x.shape[1] == 1:
        return x.flatten().detach().cpu().numpy()
    return x.detach().cpu().numpy()

In [13]:
## Training Data
t_min, t_max, t_size = 0, 5, 101
t_np = np.linspace(t_min, t_max, t_size)
t = torch.from_numpy(t_np).float().view(-1, 1).to(device)

targets = {}
targets["ic_x"] = [torch.full_like(t, 0)], torch.full_like(t, 1)
targets["ic_y"] = [torch.full_like(t, 0)], torch.full_like(t, 0)

In [14]:
## Trian model
model = PINN(layers_dim=layers, activation="tanh").to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.95)

odes = Trainer(model, optimizer, loss_fn=residual_loss, targets=targets)
losses = odes.fit(inputs=[t], n_epochs=n_epochs, scheduler=scheduler)

Epoch[10000/10000] (lr: 6.23e-06) TOTAL: 5.00e-01 RES: 4.43e-06 IC_X: 2.50e-01 IC_Y: 2.50e-01: 100%|##########| 10000/10000 [00:29<00:00, 343.67it/s]
